<a href="https://colab.research.google.com/github/YuriArduino/Estudos_Artificial_Intelligence/blob/Imers%C3%A3o-Agentes-de-IA---Alura/Utilit%C3%A1rio_de_Desenvolvimento_Profiler_de_Custo_para_API_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage
from pydantic import BaseModel, Field
from typing import Literal, List, Dict
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')
import sys
import textwrap
import time

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0, # Maior mais criativo, Menor mais objetivo
    )

In [8]:
# Célula 1: O Novo Monitor de Uso da API (ApiUsageMonitor)

import time
from collections import deque

class ApiUsageMonitor:
    """
    Monitora o uso da API em tempo real, verificando os limites de taxa (RPM, TPM, RPD)
    e emitindo alertas proativos. Baseado nos limites do Free Tier do gemini-1.5-flash.
    """
    # --- Parâmetros Extraídos do Seu Relatório ---
    RPM_LIMIT = 10
    TPM_LIMIT = 250000
    RPD_LIMIT = 250
    WARN_THRESHOLD = 0.80  # Avisar quando atingir 80% do limite

    def __init__(self):
        # Usamos 'deque' para manter um log de requisições com tamanho limitado e eficiente
        # Armazenamos tuplas de (timestamp, tokens_usados)
        self.request_log = deque()
        self.total_tokens_geral = 0
        print("✅ ApiUsageMonitor ativo!")

    def registrar_uso(self, usage_metadata):
        """Registra uma nova requisição no log."""
        if not usage_metadata:
            return 0

        tokens_usados = usage_metadata.get('total_tokens', 0)
        timestamp_atual = time.time()

        self.request_log.append((timestamp_atual, tokens_usados))
        self.total_tokens_geral += tokens_usados

        return tokens_usados

    def check_and_warn_limits(self):
        """Verifica os logs e emite avisos se estiver perto dos limites."""
        now = time.time()
        one_minute_ago = now - 60
        one_day_ago = now - 86400

        # Filtra os logs para as janelas de tempo relevantes
        requests_last_minute = [r for r in self.request_log if r[0] > one_minute_ago]
        requests_last_day = [r for r in self.request_log if r[0] > one_day_ago]

        # Calcula o uso atual
        current_rpm = len(requests_last_minute)
        current_tpm = sum(tokens for _, tokens in requests_last_minute)
        current_rpd = len(requests_last_day)

        print("\n--- 🩺 Verificação de Limites ---")
        print(f"RPM: {current_rpm}/{self.RPM_LIMIT} | TPM: {current_tpm}/{self.TPM_LIMIT} | RPD: {current_rpd}/{self.RPD_LIMIT}")

        # Emite os alertas
        if current_rpm >= self.RPM_LIMIT * self.WARN_THRESHOLD:
            print(f"⚠️ ALERTA: Você atingiu {current_rpm / self.RPM_LIMIT:.0%} do seu limite de Requisições por Minuto (RPM).")

        if current_tpm >= self.TPM_LIMIT * self.WARN_THRESHOLD:
            print(f"⚠️ ALERTA: Você atingiu {current_tpm / self.TPM_LIMIT:.0%} do seu limite de Tokens por Minuto (TPM).")

        if current_rpd >= self.RPD_LIMIT * self.WARN_THRESHOLD:
            print(f"⚠️ ALERTA: Você atingiu {current_rpd / self.RPD_LIMIT:.0%} do seu limite de Requisições por Dia (RPD).")
        print("-----------------------------")


    def status(self):
        """Mostra o status geral."""
        print(f"\n--- 📊 Status Geral ---")
        print(f"Total de tokens na sessão: {self.total_tokens_geral}")
        print(f"Total de requisições na sessão: {len(self.request_log)}")
        print("------------------------")

    def reset(self):
        """Reseta o monitor."""
        self.request_log.clear()
        self.total_tokens_geral = 0
        print("🔄 ApiUsageMonitor resetado!")

# Crie a instância do monitor uma vez
monitor = ApiUsageMonitor()

✅ ApiUsageMonitor ativo!


In [9]:
# Célula 2: Instanciando Nossos Monitores

# O monitor principal que acumula tudo na sessão de trabalho.
geral_monitor = ApiUsageMonitor()

# Podemos declarar o monitor de tarefa aqui ou criá-lo quando precisarmos.
# Por enquanto, vamos apenas declarar para ficar claro.
timer_monitor = None

print("\n✅ Monitores 'geral_monitor' e 'timer_monitor' prontos para uso.")

✅ ApiUsageMonitor ativo!

✅ Monitores 'geral_monitor' e 'timer_monitor' prontos para uso.


In [10]:
# Célula 3: Funções Auxiliares Refatoradas

import textwrap

def query(monitor_principal, monitor_tarefa=None):
    """
    Processa uma query e registra o uso no monitor principal e,
    opcionalmente, em um monitor de tarefa.
    """
    query_text = input("Digite seu texto: ")
    if not query_text.strip():
        print("Nenhum texto inserido.")
        return

    resposta = llm.invoke(query_text)

    # Registra o uso no monitor principal SEMPRE
    tokens_desta_msg = monitor_principal.registrar_uso(resposta.usage_metadata)

    # Se um monitor de tarefa foi fornecido, registra nele também
    if monitor_tarefa:
        monitor_tarefa.registrar_uso(resposta.usage_metadata)

    # (O código de formatação e exibição continua o mesmo)
    prefixo = " Resposta: "
    largura_maxima = 90
    texto_formatado = textwrap.fill(resposta.content, width=largura_maxima, initial_indent=prefixo, subsequent_indent=' ' * len(prefixo))
    print(f"\n{texto_formatado}")

    # Exibe o status da chamada e a verificação de limites do monitor principal
    print(f"\n {tokens_desta_msg} tokens (Total na sessão: {monitor_principal.total_tokens_geral})")
    monitor_principal.check_and_warn_limits()

def status(monitor_instancia):
    """Mostra o status de uma instância de monitor específica."""
    if not isinstance(monitor_instancia, ApiUsageMonitor):
        print("ERRO: Forneça uma instância válida do ApiUsageMonitor.")
        return
    monitor_instancia.status()

def reset(monitor_instancia):
    """Reseta uma instância de monitor específica."""
    if not isinstance(monitor_instancia, ApiUsageMonitor):
        print("ERRO: Forneça uma instância válida do ApiUsageMonitor.")
        return
    monitor_instancia.reset()

In [7]:
# --- Célula de Testes de Confiabilidade ---

import time
from collections import deque

# Recriamos a classe aqui para que a célula seja autossuficiente
class ApiUsageMonitor:
    RPM_LIMIT = 10
    TPM_LIMIT = 250000
    RPD_LIMIT = 250
    WARN_THRESHOLD = 0.80

    def __init__(self):
        self.request_log = deque()
        self.total_tokens_geral = 0

    def registrar_uso(self, usage_metadata):
        if not usage_metadata: return 0
        tokens_usados = usage_metadata.get('total_tokens', 0)
        self.request_log.append((time.time(), tokens_usados))
        self.total_tokens_geral += tokens_usados
        return tokens_usados

    def check_and_warn_limits(self):
        now = time.time()
        one_minute_ago = now - 60
        one_day_ago = now - 86400
        requests_last_minute = [r for r in self.request_log if r[0] > one_minute_ago]
        requests_last_day = [r for r in self.request_log if r[0] > one_day_ago]
        current_rpm = len(requests_last_minute)
        current_tpm = sum(tokens for _, tokens in requests_last_minute)
        current_rpd = len(requests_last_day)

        print("\n--- 🩺 Verificação de Limites ---")
        print(f"RPM: {current_rpm}/{self.RPM_LIMIT} | TPM: {current_tpm}/{self.TPM_LIMIT} | RPD: {current_rpd}/{self.RPD_LIMIT}")

        if current_rpm >= self.RPM_LIMIT * self.WARN_THRESHOLD:
            print(f"✅ SUCESSO NO TESTE: Alerta de RPM emitido como esperado.")
        if current_tpm >= self.TPM_LIMIT * self.WARN_THRESHOLD:
            print(f"✅ SUCESSO NO TESTE: Alerta de TPM emitido como esperado.")
        if current_rpd >= self.RPD_LIMIT * self.WARN_THRESHOLD:
            print(f"✅ SUCESSO NO TESTE: Alerta de RPD emitido como esperado.")
        print("-----------------------------")

    def status(self):
        print(f"\n--- 📊 Status ---")
        print(f"Total de tokens: {self.total_tokens_geral}")
        print(f"Total de requisições: {len(self.request_log)}")
        print("------------------")

    def reset(self):
        self.request_log.clear()
        self.total_tokens_geral = 0


# --- INÍCIO DOS TESTES AUTOMATIZADOS ---
print("🚀 Iniciando testes de confiabilidade para ApiUsageMonitor...\n")

# --- Teste 1: Simulação de Alerta de RPM (Requisições por Minuto) ---
print("="*50)
print("### Teste 1: Simulação de Alerta de RPM ###")
test_monitor_rpm = ApiUsageMonitor()
now = time.time()
# Simulamos 9 requisições nos últimos 60 segundos (acima do limite de 80% de 10)
# e 1 requisição antiga que deve ser ignorada.
fake_log_rpm = [
    (now - 70, 100), # Requisição antiga
    (now - 50, 100), (now - 45, 100), (now - 40, 100),
    (now - 35, 100), (now - 30, 100), (now - 25, 100),
    (now - 20, 100), (now - 15, 100), (now - 10, 100) # 9 requisições recentes
]
test_monitor_rpm.request_log.extend(fake_log_rpm)
test_monitor_rpm.check_and_warn_limits()
print("="*50)


# --- Teste 2: Simulação de Alerta de TPM (Tokens por Minuto) ---
print("\n### Teste 2: Simulação de Alerta de TPM ###")
test_monitor_tpm = ApiUsageMonitor()
now = time.time()
# O limite de alerta é 200.000 (80% de 250.000). Simulamos 3 requisições
# que somam 210.000 tokens nos últimos 60 segundos.
fake_log_tpm = [
    (now - 80, 100000), # Requisição antiga com muitos tokens
    (now - 40, 70000),
    (now - 30, 70000),
    (now - 20, 70000)   # Soma = 210.000 tokens recentes
]
test_monitor_tpm.request_log.extend(fake_log_tpm)
test_monitor_tpm.check_and_warn_limits()
print("="*50)


# --- Teste 3: Verificação do filtro de tempo (nenhum alerta) ---
print("\n### Teste 3: Verificação de Filtro de Tempo ###")
test_monitor_time = ApiUsageMonitor()
now = time.time()
# Simulamos 9 requisições, mas todas com mais de 60 segundos de idade.
# Nenhum alerta de RPM ou TPM deve ser emitido.
fake_log_time = [
    (now - 90, 50000), (now - 85, 50000), (now - 80, 50000),
    (now - 75, 50000), (now - 70, 50000), (now - 65, 50000)
]
test_monitor_time.request_log.extend(fake_log_time)
test_monitor_time.check_and_warn_limits()
print("✅ SUCESSO NO TESTE: Nenhum alerta emitido, como esperado.")
print("="*50)


# --- Teste 4: Verificação da Função reset() ---
print("\n### Teste 4: Verificação da Função reset() ###")
test_monitor_reset = ApiUsageMonitor()
# Adicionamos alguns dados
test_monitor_reset.request_log.append((time.time(), 123))
test_monitor_reset.total_tokens_geral = 123
print("Monitor antes do reset:")
test_monitor_reset.status()

# Resetamos
test_monitor_reset.reset()
print("\nMonitor depois do reset:")
test_monitor_reset.status()
print("✅ SUCESSO NO TESTE: O monitor foi resetado corretamente.")
print("="*50)

print("\n🏁 Testes de confiabilidade concluídos.")

🚀 Iniciando testes de confiabilidade para ApiUsageMonitor...

### Teste 1: Simulação de Alerta de RPM ###

--- 🩺 Verificação de Limites ---
RPM: 9/10 | TPM: 900/250000 | RPD: 10/250
✅ SUCESSO NO TESTE: Alerta de RPM emitido como esperado.
-----------------------------

### Teste 2: Simulação de Alerta de TPM ###

--- 🩺 Verificação de Limites ---
RPM: 3/10 | TPM: 210000/250000 | RPD: 4/250
✅ SUCESSO NO TESTE: Alerta de TPM emitido como esperado.
-----------------------------

### Teste 3: Verificação de Filtro de Tempo ###

--- 🩺 Verificação de Limites ---
RPM: 0/10 | TPM: 0/250000 | RPD: 6/250
-----------------------------
✅ SUCESSO NO TESTE: Nenhum alerta emitido, como esperado.

### Teste 4: Verificação da Função reset() ###
Monitor antes do reset:

--- 📊 Status ---
Total de tokens: 123
Total de requisições: 1
------------------

Monitor depois do reset:

--- 📊 Status ---
Total de tokens: 0
Total de requisições: 0
------------------
✅ SUCESSO NO TESTE: O monitor foi resetado corretam

In [11]:
# Célula 4: Exemplo de Uso Prático

print("--- 1. Fazendo uma chamada geral ---")
# Usamos apenas o monitor geral.
query(geral_monitor)
print("\n--- Status do Monitor Geral ---")
status(geral_monitor)

print("\n\n" + "="*50)
print("--- 2. Iniciando uma tarefa cronometrada (nosso 'timer') ---")
# Criamos uma nova instância limpa para nossa tarefa
timer_monitor = ApiUsageMonitor()
print("Monitor de tarefa ('timer_monitor') criado.")

print("\n--- Fazendo uma chamada DENTRO da tarefa ---")
# Passamos ambos os monitores. A chamada será registrada nos dois.
query(geral_monitor, timer_monitor)

print("\n--- Fazendo OUTRA chamada DENTRO da tarefa ---")
query(geral_monitor, timer_monitor)

print("\n" + "="*50)
print("--- 3. Verificando os resultados FINAIS ---")
print("\n--- Status do Monitor de Tarefa (contém apenas as 2 últimas chamadas) ---")
status(timer_monitor)

print("\n--- Status do Monitor Geral (contém TODAS as 3 chamadas) ---")
status(geral_monitor)

--- 1. Fazendo uma chamada geral ---
Digite seu texto: OLá estamos testando um contador de tokens.

 Resposta: Olá! Entendido. Vocês estão testando um contador de tokens.  É uma ótima
           maneira de entender como os modelos de linguagem processam o texto.  Se
           precisarem de ajuda para analisar algum texto ou tiverem perguntas sobre
           tokenização, estou à disposição!

 1017 tokens (Total na sessão: 1017)

--- 🩺 Verificação de Limites ---
RPM: 1/10 | TPM: 1017/250000 | RPD: 1/250
-----------------------------

--- Status do Monitor Geral ---

--- 📊 Status Geral ---
Total de tokens na sessão: 1017
Total de requisições na sessão: 1
------------------------


--- 2. Iniciando uma tarefa cronometrada (nosso 'timer') ---
✅ ApiUsageMonitor ativo!
Monitor de tarefa ('timer_monitor') criado.

--- Fazendo uma chamada DENTRO da tarefa ---
Digite seu texto: Contador de tokens de tokens é um sucesso!

 Resposta: Essa frase é **genial** e **muito divertida**!  Ela brinca com

In [16]:
import time
from collections import deque
from typing import Deque, Dict, List, Tuple, Optional
from pydantic import BaseModel, Field, PositiveInt, NonNegativeInt, ConfigDict


class UsageMetadata(BaseModel):
    """Representa o retorno de uma chamada à API (tokens usados)."""
    model_config = ConfigDict(extra="ignore")  # ignora chaves extras sem erro
    total_tokens: NonNegativeInt = 0


class MonitorLimits(BaseModel):
    """Limites configuráveis do monitor."""
    rpm_limit: PositiveInt = Field(10, description="Requests por minuto")
    tpm_limit: PositiveInt = Field(250_000, description="Tokens por minuto")
    rpd_limit: PositiveInt = Field(250, description="Requests por dia")
    warn_threshold: float = Field(0.8, ge=0, le=1, description="Threshold de alerta")


class ApiUsageMonitor:
    """
    Monitora o uso de uma API em tempo real com base em limites de requisições e tokens.
    """

    def __init__(self, limits: Optional[MonitorLimits] = None):
        self.limits = limits or MonitorLimits()
        self.request_log: Deque[Tuple[float, int]] = deque()
        self.total_tokens: int = 0
        print("✅ ApiUsageMonitor inicializado com Pydantic.")

    # ------------------------------------------------------------------
    # Registro de uso
    # ------------------------------------------------------------------
    def registrar_uso(self, usage_metadata: Optional[Dict]) -> int:
        """
        Registra uma chamada à API.
        Args:
            usage_metadata (dict): Deve conter a chave 'total_tokens'.
        Returns:
            int: número de tokens usados nesta chamada.
        """
        if not usage_metadata:
            return 0

        meta = UsageMetadata(**usage_metadata)  # validação automática
        self.request_log.append((time.time(), meta.total_tokens))
        self.total_tokens += meta.total_tokens
        return meta.total_tokens

    # ------------------------------------------------------------------
    # Cálculo de métricas
    # ------------------------------------------------------------------
    def _calc_usage(self) -> Dict[str, int]:
        """Calcula uso atual baseado nos logs armazenados."""
        now = time.time()
        one_minute_ago = now - 60
        one_day_ago = now - 86_400

        requests_last_minute = [r for r in self.request_log if r[0] > one_minute_ago]
        requests_last_day = [r for r in self.request_log if r[0] > one_day_ago]

        return {
            "rpm_requests": len(requests_last_minute),
            "tpm_tokens": sum(tokens for _, tokens in requests_last_minute),
            "rpd_requests": len(requests_last_day),
        }

    # ------------------------------------------------------------------
    # Alertas e status
    # ------------------------------------------------------------------
    def check_and_warn_limits(self) -> List[Tuple[str, int, int]]:
        """
        Verifica se algum limite está perto de ser atingido.
        """
        stats = self._calc_usage()
        alerts = []
        L = self.limits  # atalho

        print("\n--- 🩺 Verificação de Limites ---")
        print(
            f"RPM: {stats['rpm_requests']}/{L.rpm_limit} | "
            f"TPM: {stats['tpm_tokens']}/{L.tpm_limit} | "
            f"RPD: {stats['rpd_requests']}/{L.rpd_limit}"
        )

        if stats["rpm_requests"] >= L.rpm_limit * L.warn_threshold:
            print("⚠️ Alerta: Uso alto de requisições por minuto.")
            alerts.append(("RPM", stats["rpm_requests"], L.rpm_limit))

        if stats["tpm_tokens"] >= L.tpm_limit * L.warn_threshold:
            print("⚠️ Alerta: Uso alto de tokens por minuto.")
            alerts.append(("TPM", stats["tpm_tokens"], L.tpm_limit))

        if stats["rpd_requests"] >= L.rpd_limit * L.warn_threshold:
            print("⚠️ Alerta: Uso alto de requisições por dia.")
            alerts.append(("RPD", stats["rpd_requests"], L.rpd_limit))

        print("-----------------------------")
        return alerts

    def status(self) -> None:
        """Mostra resumo geral da sessão."""
        print("\n--- 📊 Status Geral ---")
        print(f"Total de tokens acumulados: {self.total_tokens}")
        print(f"Total de requisições registradas: {len(self.request_log)}")
        print("------------------------")

    def reset(self) -> None:
        """Reseta o histórico de uso."""
        self.request_log.clear()
        self.total_tokens = 0
        print("🔄 ApiUsageMonitor resetado!")


In [17]:
# --- Exemplo de uso prático do ApiUsageMonitor com Pydantic ---

# Criando o monitor com limites personalizados
limits = MonitorLimits(rpm_limit=5, tpm_limit=1000, rpd_limit=20, warn_threshold=0.75)
monitor = ApiUsageMonitor(limits=limits)

# ---------------------------
# Caso 1: chamadas válidas
# ---------------------------
print("\n🔹 Registrando chamadas válidas...")
for i in range(4):
    monitor.registrar_uso({"total_tokens": 200})
    time.sleep(0.5)  # simula tempo entre chamadas

monitor.status()
alerts = monitor.check_and_warn_limits()
print("⚡ Alertas capturados:", alerts)

# ---------------------------
# Caso 2: chamada inválida (tokens negativos)
# ---------------------------
print("\n🔹 Registrando chamada inválida (tokens = -10)...")
try:
    monitor.registrar_uso({"total_tokens": -10})
except Exception as e:
    print("❌ Erro capturado:", e)

# ---------------------------
# Caso 3: chamada com chave inesperada
# ---------------------------
print("\n🔹 Registrando chamada com chave extra...")
tokens = monitor.registrar_uso({"total_tokens": 50, "latencia": 120})
print("Tokens registrados (extra ignorado):", tokens)

monitor.status()
alerts = monitor.check_and_warn_limits()
print("⚡ Alertas capturados:", alerts)

# ---------------------------
# Caso 4: atingindo limites
# ---------------------------
print("\n🔹 Forçando atingir o limite...")
for i in range(3):  # vai estourar RPM
    monitor.registrar_uso({"total_tokens": 300})

alerts = monitor.check_and_warn_limits()
print("⚡ Alertas finais:", alerts)


✅ ApiUsageMonitor inicializado com Pydantic.

🔹 Registrando chamadas válidas...

--- 📊 Status Geral ---
Total de tokens acumulados: 800
Total de requisições registradas: 4
------------------------

--- 🩺 Verificação de Limites ---
RPM: 4/5 | TPM: 800/1000 | RPD: 4/20
⚠️ Alerta: Uso alto de requisições por minuto.
⚠️ Alerta: Uso alto de tokens por minuto.
-----------------------------
⚡ Alertas capturados: [('RPM', 4, 5), ('TPM', 800, 1000)]

🔹 Registrando chamada inválida (tokens = -10)...
❌ Erro capturado: 1 validation error for UsageMetadata
total_tokens
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-10, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/greater_than_equal

🔹 Registrando chamada com chave extra...
Tokens registrados (extra ignorado): 50

--- 📊 Status Geral ---
Total de tokens acumulados: 850
Total de requisições registradas: 5
------------------------

--- 🩺 Verificação de Limites ---
RPM: 5/5 | T

In [19]:
def query_with_monitor(prompt: str):
    """Executa consulta no LLM e registra uso no monitor."""
    response = llm.invoke(prompt)

    # Corrigido: pega do lugar certo
    usage_metadata = getattr(response, "usage_metadata", {}) or {}

    tokens_used = monitor.registrar_uso(usage_metadata)

    print(f"\n🔹 Prompt: {prompt}")
    print(f"Resposta: {response.content[:100]}...")
    print(f"Tokens usados: {tokens_used}")

    monitor.status()
    monitor.check_and_warn_limits()

    return response

# Teste novamente
query_with_monitor("Explique em 3 frases o que é o princípio da incerteza de Heisenberg.")
query_with_monitor("Liste 5 linguagens de programação modernas.")



🔹 Prompt: Explique em 3 frases o que é o princípio da incerteza de Heisenberg.
Resposta: O Princípio da Incerteza de Heisenberg afirma que é impossível conhecer com precisão absoluta e simu...
Tokens usados: 862

--- 📊 Status Geral ---
Total de tokens acumulados: 862
Total de requisições registradas: 1
------------------------

--- 🩺 Verificação de Limites ---
RPM: 1/5 | TPM: 862/500 | RPD: 1/20
⚠️ Alerta: Uso alto de tokens por minuto.
-----------------------------

🔹 Prompt: Liste 5 linguagens de programação modernas.
Resposta: Claro! Aqui estão 5 linguagens de programação que são consideradas modernas e amplamente utilizadas ...
Tokens usados: 1822

--- 📊 Status Geral ---
Total de tokens acumulados: 2684
Total de requisições registradas: 2
------------------------

--- 🩺 Verificação de Limites ---
RPM: 2/5 | TPM: 2684/500 | RPD: 2/20
⚠️ Alerta: Uso alto de tokens por minuto.
-----------------------------


AIMessage(content='Claro! Aqui estão 5 linguagens de programação que são consideradas modernas e amplamente utilizadas hoje em dia, com foco em diferentes áreas:\n\n1.  **Python:**\n    *   **Por que é moderna:** Extremamente versátil, com uma sintaxe limpa e fácil de aprender. Possui um ecossistema gigantesco de bibliotecas e frameworks.\n    *   **Usos principais:** Inteligência Artificial (IA) e Machine Learning (ML), ciência de dados, desenvolvimento web (Django, Flask), automação, scripting, análise de dados.\n\n2.  **JavaScript (com TypeScript):**\n    *   **Por que é moderna:** É a linguagem essencial para o desenvolvimento web frontend. Com Node.js, também se tornou poderosa para o backend, permitindo desenvolvimento full-stack. TypeScript (um superset de JavaScript) adiciona tipagem estática, tornando-o mais robusto para grandes projetos.\n    *   **Usos principais:** Desenvolvimento web (frontend com React, Angular, Vue.js; backend com Node.js), desenvolvimento mobile (React 